# 참고2. KoGPT 모델 추론하기

- 학습 내용
    - Hard Prompt 방식을 활용하여 GPT 모델에게 생성 요약을 시킵니다.
    - [8-bit Matrix Multiplication](https://huggingface.co/blog/hf-bitsandbytes-integration) 를 활용하여 큰 언어모델을 작은 GPU 메모리에서 실행합니다.
- 환경 필요사항
    - KaKao-Brain에서 공개한 Ko-GPT3 모델을 활용
    - 일종의 경량화 버전인 float-16 버전의 모델을 활용하여도 Colab 무료 사용으로는 실습 제한
    - 원활한 실습을 위해서는 월 9.99 달러의 Colab Pro를 활용하는것을 권장
<!-- ![nn](prompt-to-generate.jpg) -->

In [ ]:
# 구글 드라이브와 연동합니다. 권한 허용이 필요합니다.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 라이브러리를 설치합니다. bitsandbytes는 load_in_8bit옵션을 사용하려면 설치되어야합니다.
%pip install -q transformers accelerate
%pip install -q bitsandbytes

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader,Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM 
from tqdm.auto import tqdm
import os, gc
from datetime import datetime, timezone, timedelta

## 모델, 토크나이저 불러오기

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
)

- load_in_8bit 옵션을 사용하면 [bitsandbytes](https://github.com/TimDettmers/bitsandbytes)를 이용한 8bit 연산 방식으로 불러옵니다.
- 불러올 모델은 float16을 사용하므로 dtype을 지정해주면 불러오는 과정에 사용하는 메모리를 절약할 수 있습니다.
- device_map = 'auto'를 사용하여 weight를 불러오면서 바로 GPU로 올리면 CPU 메모리가 작아도 실행 할 수 있습니다.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt', revision = 'KoGPT6B-ryan1.5b-float16',
    torch_dtype = torch.float16,
    load_in_8bit = True,
    device_map = 'auto',
)

In [ ]:
pd.DataFrame([
    (param.dtype, param.shape, param.device, param.requires_grad, name)
    for name, param in peft_model.named_parameters()
], columns=['dtype', 'shape', 'device', 'requires_grad', 'name'])

## Hard Prompt 데이터 셋 작성
- `{본문} 한줄 요약:` 형태로 input을 만들고 모델이 뒷부분을 생성하게 합니다.
- 모델은 따로 학습하지 않으므로 `한줄 요약:`이라는 prompt를 통해서만 요약과제인 것을 알 수 있습니다.
- 길이가 서로 다른 샘플을 하나의 배치로 만들기 위해 collate함수를 작성합니다.
- GPTJ는 문장의 오른쪽 끝부터 생성하는 autoregressive 모델이므로 오른쪽 끝이 같아야합니다.
- 배치 내에서 가장 긴 샘플에 맞춰서 left padding 합니다. 즉, `[PAD][PAD]...[PAD] {본문} 한줄 요약:` 형태가 됩니다.
- `[PAD]`는 정보를 담고 있지 않으므로 attention_mask를 0으로 설정해서 무시합니다.

In [ ]:
class SummaryTestDataset(Dataset):
    def __init__(self, data_path, tokenizer):
        self._data = pd.read_csv(data_path)
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self._data)
    
    def __getitem__(self, idx):
        row = self._data.iloc[idx]
        prompt = "{text} 한줄 요약:"
        input_text = prompt.format(text=row['text'])
        input_encoding = self.tokenizer(input_text)

        result = {
            'input_ids': input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
        }
        
        return result

    def _left_pad(self, sequence, value, max_len):
        return [value] * (max_len - len(sequence)) + sequence

    def collate_fn(self, batch, device='cuda'):
        input_length = max(len(row['input_ids']) for row in batch)

        input_ids = [
            self._left_pad(row['input_ids'], self.tokenizer.pad_token_id, input_length)
            for row in batch
        ]
        attention_mask = [
            self._left_pad(row['attention_mask'], 0, input_length)
            for row in batch
        ]

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long, device=device),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long, device=device),
        }

In [ ]:
test_path = '/content/drive/MyDrive/GPT_Competition/test.csv'
test_set = SummaryTestDataset(test_path, tokenizer)
test_loader = DataLoader(test_set, batch_size=4, num_workers=0, shuffle=False, collate_fn=test_set.collate_fn)

## 생성 요약하기

- [generate](https://huggingface.co/docs/transformers/v4.27.1/en/main_classes/text_generation#transformers.GenerationMixin.generate) 메소드로 이어질 문장을 생성합니다.
- colab 기준 1시간 정도 소요됩니다.
- generate 생성방법에 대한 설명은 커뮤니티를 참조하세요.
    - max_length: (프롬프트 포함) 문장 최대 토큰 수
    - max_new_tokens: 생성할 최대 토큰 수
    - min_new tokens: 생성할 최소 토큰 수
    - num_beams: beam_search에 사용할 beam 수 입니다. beam search 하지 않는 경우 1입니다.
    - do_sample: 랜덤 샘플합니다. False면 좋은 값만 선택합니다.
    - temperature: 높을수록 더 기존 확률을 무시하고 샘플합니다.
    - top_k: 각 토큰을 확률이 높은 k개 중에서만 선택합니다.
    - top_p: 각 토큰을 확률순으로 누적합이 p 이하인 범위에서만 선택합니다.
    - [GenerationConfig](https://huggingface.co/docs/transformers/v4.27.1/en/main_classes/text_generation#transformers.GenerationConfig)에서 더 확인 할 수 있습니다.

In [ ]:
def predict():
    preds = []
    model.eval()
    for batch_idx, batch in enumerate(tqdm(test_loader)):
        with torch.no_grad():
            with torch.amp.autocast('cuda'):
                generated = model.generate(
                    input_ids = batch['input_ids'],
                    attention_mask = batch['attention_mask'],

                    pad_token_id = tokenizer.pad_token_id,
                    max_new_tokens = 100,
                    do_sample = False,
                    num_beams = 1,
                    num_beam_groups = 1,
                    use_cache = True,
                )
            prompted_length = batch['input_ids'].size(-1)
            summary_tokens = generated[:, prompted_length:]
            summary = tokenizer.batch_decode(summary_tokens, skip_special_tokens=True)
            preds.extend(summary)
            print(*summary, sep='\n----------\n',end='\n========\n')
    return preds

preds = predict()

In [ ]:
test_df = pd.read_csv(test_path)
test_df['summary'] = preds

In [ ]:
# 현재 시간으로 이름붙인 제출파일을 생성합니다.
TIME_SERIAL = datetime.now(timezone(timedelta(hours=9))).strftime("%y%m%d-%H%M%S")
SUBMISSION_PATH = f'/content/drive/MyDrive/GPT_Competition/exp_{TIME_SERIAL}.csv'
test_df[['id', 'summary']].to_csv(SUBMISSION_PATH, index=False)
print(SUBMISSION_PATH)

In [ ]:
# 자동으로 세션을 종료하고 싶을때 사용하세요.
# from google.colab import runtime
# runtime.unassign()